In [4]:
import googleapiclient.discovery
import googleapiclient.errors
from datetime import datetime
import psycopg2
import os
from dotenv import load_dotenv

In [5]:
api_service_name = 'youtube'
api_version = 'v3'
load_dotenv()
DEVELOPER_KEY = os.getenv('DEVELOPER_KEY')

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY
)

In [7]:
channels_list = ['@Notorious_3cm']
channels = []

request = youtube.search().list(
    part='snippet',
    q=channels_list[0],
    type='channel',
    maxResults=1
)


for channel_name in channels_list:
    
    try:
        response = request.execute()

        if 'items' in response:
            channel_id = response['items'][0]['id']['channelId']

            channel_response = youtube.channels().list(
                part='statistics',
                id=channel_id
            ).execute()
            
            if 'items' in channel_response:
                subscriber_count = channel_response['items'][0]['statistics']['subscriberCount']
            
            channels.append({'channel_id': channel_id,
                            'channel_name':channel_name,
                            'subscriberCount':subscriber_count,
                            'pol_affiliation':'neutral'}
                            )
            
            print(f'Channel {channel_name} has been found. id={channel_id}')
        else:
            print('Channel is not found')
    except Exception as e:
        print(f'Error searchign for Channel {channel_name}')


print(channels)

Channel @Notorious_3cm has been found. id=UCi2GvcaxZCN-61a0co8Smnw
[{'channel_id': 'UCi2GvcaxZCN-61a0co8Smnw', 'channel_name': '@Notorious_3cm', 'subscriberCount': '884000', 'pol_affiliation': 'neutral'}]


In [ ]:
dbname_var = os.getenv('DATABASE_NAME')
user_var = os.getenv('DATABASE_USER')
password_var = os.getenv('DATABASE_PASSWORD')
host_var = os.getenv('DATABASE_HOST')
port_var = os.getenv('DATABASE_PORT')

In [90]:
conn = psycopg2.connect("dbname=dbname_var user=user_var password=password_var host=host_var port=port_var")
curr = conn.cursor()

try:
    for channel in channels:
        curr.execute("""
                    INSERT into channels (channel_id,channel_name,subscriberCount,pol_affiliation)
                    VALUES (%s,%s,%s,%s)
                    """,
                    (channel['channel_id'],channel['channel_name'],
                    channel['subscriberCount'],channel['pol_affiliation']))
        conn.commit()
        print('Data inserted succesfully')
except psycopg2.Error as e:
    print(f'Error inserting data: {e}')
    conn.rollback()
finally:
    curr.close()
    conn.close()

Data inserted succesfully


In [5]:
start_date = datetime.strptime('2024-05-17T00:00:00Z','%Y-%m-%dT%H:%M:%SZ')

videos = []

for channel_info in channels:
    channel_id = channel_info['channel_id']
    channel_name = channel_info['channel_name']
    try:    
        request = youtube.search().list(
            part='snippet',
            channelId=channel_id,
            type='video',
            q='黑箱',
            maxResults=1000
        )

        response = request.execute()
 
        if 'items' in response:
            for item in response['items']: 
                video_id = item['id']['videoId']
                video_title = item['snippet']['title']
                published_at = item['snippet']['publishedAt']
                
                video_response = youtube.videos().list(
                    part='statistics',
                    id=video_id
                ).execute()
                
                if 'items' in video_response:
                    statistics = video_response['items'][0]['statistics']
                    view_count = statistics['viewCount']
                    like_count = statistics['likeCount']
                    comment_count = statistics['commentCount']
                else:
                    view_count = 0
                    like_count = 0
                    comment_count = 0                   
                    
                videos.append({'channel_id': channel_id,
                               'video_id': video_id, 
                               'video_title': video_title,
                               'view_count': view_count,
                               'like_count': like_count,
                               'comment_count': comment_count,
                               'published_at': published_at})
                print(f'Video on {channel_name} has been found. id={video_id}')
        else:
            print('Videos are not found')
    except Exception as e:
        print(f'Error searchign for videos on channel {channel_name}')

print(videos)

Video on @Notorious_3cm has been found. id=gtIz_u6XP90
Video on @Notorious_3cm has been found. id=gY2rjam01nY
Video on @Notorious_3cm has been found. id=8NEjLeso8fQ
Video on @Notorious_3cm has been found. id=Q-Dr1DcgP14
Video on @Notorious_3cm has been found. id=WnoVjI0utV8
Video on @Notorious_3cm has been found. id=G7MqPpvzYg0
Video on @Notorious_3cm has been found. id=wka_HMVQbog
Video on @Notorious_3cm has been found. id=z_v4jgoWJd0
Video on @Notorious_3cm has been found. id=02wvF3QcjzY
Video on @Notorious_3cm has been found. id=XgCxt1HyyQM
Video on @Notorious_3cm has been found. id=BIUSJsIk6AE
Video on @Notorious_3cm has been found. id=dNPT9-WTPEs
Video on @Notorious_3cm has been found. id=x7Eb0NTHsPg
Video on @Notorious_3cm has been found. id=zcifxrFFSG4
Video on @Notorious_3cm has been found. id=7NcpP3rcGPw
Video on @Notorious_3cm has been found. id=yktq7o-GdBo
Video on @Notorious_3cm has been found. id=AVpzPfOTSO8
Video on @Notorious_3cm has been found. id=sm0R1xU19lM
Video on @

In [97]:
conn = psycopg2.connect("dbname=heixiang user=kuan password=112233zxc host=localhost port=5432")
curr = conn.cursor()
try:
    for video in videos:
        curr.execute("""
                    INSERT into videos (channel_id, video_id, video_title, view_count, like_count, comment_count, published_at)
                    VALUES (%s,%s,%s,%s,%s,%s,%s)
                    """,
                    (video['channel_id'],video['video_id'],video['video_title'],
                    video['view_count'],video['like_count'],video['comment_count'],video['published_at']))
        conn.commit()
        print('Data inserted succesfully')
except psycopg2.Error as e:
    print(f'Error inserting data: {e}')
    conn.rollback()
finally:
    curr.close()
    conn.close()

Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully


In [16]:
comments = []
for video in videos:
    video_id = video['video_id']

    try:
        request = youtube.commentThreads().list(
            part='snippet', videoId=video_id,
            maxResults=100
        )

        response = request.execute()

        
        if 'items' in response:
            for item in response['items']:
                top_comment = item['snippet']['topLevelComment']['snippet']
                if top_comment['likeCount'] > 0:
                    comments.append({'video_id': video_id,
                                    'comment_id': item['snippet']['topLevelComment']['id'],
                                    'comment_content': top_comment['textDisplay'],
                                    'like_count': top_comment['likeCount'],
                                    'reply_count': item['snippet']['totalReplyCount'],
                                    'published_at': top_comment['publishedAt']
                                    })
                    print(f'Comment under video {video_id} has been found. id={item['snippet']['topLevelComment']['id']}')
        else:
            print('Comments are not found')
    except Exception as e:
                print(f'Error searchign for comments under video {video_id}')

                        
print(comments)

Comment under video gtIz_u6XP90 has been found. id=Ugyy5lZW4GT-s0iN-pZ4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugwo84m4cTh4Yv3usWl4AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgxOFTtaSIL9yu8ATXZ4AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgwBor4M37wy1ntBfUZ4AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgzmZ6Ij-87Qq7DWe5d4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugxn1KAN9VW7-Aua5-B4AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgzdJMgQ_4Yhg3ab06x4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugy4jPtRJzkHS5HwbXB4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugw4EXpesO9rwNyFsHN4AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgygSw_Qg8HpC7X-iuV4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugyhca_mZ_fTs8GahA54AaABAg
Comment under video gtIz_u6XP90 has been found. id=UgwdDUSWuw1cbQ4HKWR4AaABAg
Comment under video gtIz_u6XP90 has been found. id=Ugw1q_XAHkkKj

In [17]:
conn = psycopg2.connect("dbname=heixiang user=kuan password=112233zxc host=localhost port=5432")
curr = conn.cursor()
try:
    for comment in comments:
        curr.execute("""
                    INSERT into comments (comment_id, video_id, published_at, like_count, reply_count, comment_content)
                    VALUES (%s,%s,%s,%s,%s,%s)
                    """,
                    (comment['comment_id'],comment['video_id'],comment['published_at'],
                    comment['like_count'],comment['reply_count'],comment['comment_content']))
        conn.commit()
        print('Data inserted succesfully')
except psycopg2.Error as e:
    print(f'Error inserting data: {e}')
    conn.rollback()
finally:
    curr.close()
    conn.close()

Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserted succesfully
Data inserte